In [70]:
import numpy as np

def loadData():
    import csv

    var = []

    dataFile = open('CS170_SMALLtestdata__80.txt', 'r')
    data = csv.reader(dataFile, delimiter=' ', skipinitialspace=True)
    for row in data:
        var.append(row)
    dataFile.close()

    return var

X = np.array(loadData())
X = X.astype(np.float)

In [86]:
def nearestNeighbor(data, current_set, feature_to_add):
    total_set = current_set
    total_set.append(feature_to_add)
    total_set.insert(0, 0)
    data = data[:, total_set]
    m = len(data)
    number_correctly_classified = 0

    for i in range(m):
        object_to_classify = data[i,1:]
        label_object_to_classify = data[i,0]

        nearest_neighbor_distance = float('inf')
        nearest_neighbor_location = float('inf')
        nearest_neighbor_label = 0
        for j in range(m):
            if j != i:
                distance = np.sqrt(np.sum(object_to_classify - data[j,1:])**2)
                if distance < nearest_neighbor_distance:
                    nearest_neighbor_distance = distance
                    nearest_neighbor_location = j
                    nearest_neighbor_label = data[j,0]
        if label_object_to_classify == nearest_neighbor_label:
            number_correctly_classified += 1

        print("Object ", i, " is class ", nearest_neighbor_label)
        print("Its nearest neighbor is ", nearest_neighbor_location, " which is in class ", nearest_neighbor_label)
    
    accuracy = number_correctly_classified / m
    return accuracy


In [87]:
acc = nearestNeighbor(X, [1], 3)
print("This is acc ", acc)

Object  0  is class  2.0
Its nearest neighbor is  270  which is in class  2.0
Object  1  is class  2.0
Its nearest neighbor is  240  which is in class  2.0
Object  2  is class  2.0
Its nearest neighbor is  266  which is in class  2.0
Object  3  is class  2.0
Its nearest neighbor is  95  which is in class  2.0
Object  4  is class  1.0
Its nearest neighbor is  147  which is in class  1.0
Object  5  is class  2.0
Its nearest neighbor is  12  which is in class  2.0
Object  6  is class  2.0
Its nearest neighbor is  21  which is in class  2.0
Object  7  is class  1.0
Its nearest neighbor is  79  which is in class  1.0
Object  8  is class  2.0
Its nearest neighbor is  209  which is in class  2.0
Object  9  is class  2.0
Its nearest neighbor is  190  which is in class  2.0
Object  10  is class  2.0
Its nearest neighbor is  246  which is in class  2.0
Object  11  is class  2.0
Its nearest neighbor is  4  which is in class  2.0
Object  12  is class  2.0
Its nearest neighbor is  272  which is in 